# Imports and Advanced Data Storage
To add even more functionality to smart contracts, you can import the exported functions of other smart contracts. You can also store complex data in the form of Python objects, which is useful for storing things like lists, tuples, and dates. Here's how.

In [10]:
def import_this():
    @export
    def dumb_func():
        return 4
    
def to_import():
    import import_this
    @export
    def dumber_func():
        return import_this.dumb_func()

In [11]:
from contracting.client import ContractingClient
client = ContractingClient(signer='stu')
client.flush()
client.submit(import_this)
client.submit(to_import)

In [12]:
ti_contract = client.get_contract('to_import')
ti_contract.dumber_func()

4

Easy! Notice that you can only import entire contracts. You cannot use `from a import b` notation to import just a singular function from a smart contract. Furthermore, dynamic imports will be added at a later time to allow importing a contract by name that can be used as an argument.

## Advanced Data
Imagine a game where you can buy or sell pixels. Each pixel has an X and a Y coordinate. At each pixel, we want to store the owner, the price, and the color. While this might sound complicated, it is actually extremely straight forward to do.

All `Hash` objects have the ability to store up to 16 dimensions of information in a key that a max of 1024 bytes in size. The stored object is just JSON using the standard Python JSON encoder and decoder, so you can store things such as lists, sets, etc. At this point in time, there is no limit on how large the value for a key can be, but it will be capped in the future. The current floating figure is 256 bytes to 1024 bytes, so design your contracts accordingly.

Unlike normal Python dictionaries, which `Hash` objects are similar to, you can add different dimensions to your `Hash` object without an issue. For example:

```
h = Hash()
h['one'] = 15
h['one', 'two'] = 20
h['two'] = 25
h['a', 'b', 'c', 'd', 'e', 'f', 'g'] = 6
```

Let's try to build that pixel application.

In [4]:
def coin():
    balances = Hash(default_value=0)
    token_name = 'Stubucks'
    token_symbol = 'SBX'
    
    @construct
    def seed():
        # Whoever creates this smart contract is minted 1,000,000 tokens
        balances[ctx.caller] = 1000000
        
    @export
    def transfer(amount, to):
        # Make sure that the person calling this function has the amount they are trying to transfer
        assert balances[ctx.caller] >= amount, "You don't have enough to spend!"
        
        # If so, deduct from their account and send to who they want to send to
        balances[ctx.caller] -= amount
        balances[to] += amount
        
    @export
    def allow(amount, spender):
        # This creates a 'subaccount' to allow the spender to transfer from our account a certain amount
        balances[ctx.caller, spender] = amount
        
    @export
    def spend_on_behalf(amount, owner, to):
        # We make sure the subaccount has enough coins to spend
        assert balances[owner, ctx.caller] >= amount, "You can't spend that!"
        
        # If so, deduct from the amount that the subaccount can spend
        balances[owner, ctx.caller] -= amount
        
        # And then make the transfer
        balances[owner] -= amount
        balances[to] += amount

def pixel_game():
    import coin
    
    # We set the default value of dictionary values to None for testing if they exist
    pixels = Hash(default_value=None)
    
    # These constants can never be changed. If you want mutable variables, use Variable objects and provide getters
    # and setters.
    max_x = 256
    max_y = 256
    
    # Just a palette to simplify this
    color_min = 0
    color_max = 16
    
    @export
    def buy_pixel(x, y, color, amount):
        assert x < max_x and x >= 0, 'X out of bounds!'
        assert y < max_y and y >= 0, 'Y out of bounds!'
        
        assert color < color_max and color >= color_min, 'Color is out of bounds!'
        
        # If we make it to here, we can access the pixel.
        pixel = pixels[x, y]
        
        # If it is None, it's never been bought before, so we can buy it outright
        if pixel is None:
            # Take the coins and store it in the pixel game's account
            overwrite_pixel(x, y, color, amount, ctx.caller)
            
        else:
            # Otherwise, the pixel is a dictionary, so we can access it like such
            assert amount > pixel['amount'], 'You must pay at least {} to purchase.'.format(pixel['amount'])
            overwrite_pixel(x, y, color, amount, ctx.caller)
            
    def overwrite_pixel(x, y, color, amount, owner):
        coin.spend_on_behalf(amount=amount, owner=ctx.caller, to=ctx.this)
            
        pixels.set[x, y] = {
            'owner': owner,
            'amount': amount,
            'color': color
        }


To keep things simple, let's just assume that we can buy pixels. If you see `overwrite_pixel`, you'll notice that we are setting the X and Y coordinates to a Python dictionary rather than just a primitive type. When we access the database again to pull it out, it is decoded into a Python dictionary again, so you can access it like how you are used to.

Let's try to buy a pixel!

In [5]:
client.submit(coin)
client.submit(pixel_game)

In [6]:
coin_contract = client.get_contract('coin')
pixel_contract = client.get_contract('pixel_game')

In [7]:
coin_contract.balances['stu'] # Let's make sure we have the coins to make a pixel purchase.

1000000

In [8]:
coin_contract.allow(amount=1, spender='pixel_game')
pixel_contract.buy_pixel(x=10, y=10, color=5, amount=1)

In [9]:
coin_contract.balances['stu'] # The balance has deducted properly

999999

In [10]:
pixel_contract.pixels[10, 10] # Now we can access the information and recieve the dictionary back

{'owner': 'stu', 'amount': 1, 'color': 5}

In [11]:
type(pixel_contract.pixels[10, 10]) # Proof the dictionary is in fact a dictionary!

dict